# Setup

In [ ]:
# Experiment Parameters

# Experiment conditions
use_all_rewriter_rules = True
convert_bigraph = True
not_representation = "X"
not_placement = "end"
gate_layers = ["H", "Rx"]
var_layers = 1
parameterize_nouns = False
# independent_sublayer = False

# load params
train_params = True
load_params = True
# param_vals = [0.6931585473443279, 1.0462424956825283, 1.234537983904686, -2.90716906835752, 1.5753587952252757, -2.4968837592255424, 1.6118961289272027, -1.016327328767516, -0.3142432619217712, 1.208915586540799, -1.1801888411140051, -0.4286556908790561, 3.3779520000989214, 0.04286513605390508, 0.5295399156230165, -1.7699845136573933, 0.5233673239374572, -1.748569530235187, -0.26823991786760143, 1.813789372735753, -2.9079130258111228, -0.6129166776514257, 0.3401131125827421, -0.4365025178512618, 1.008439272978763, 0.06192528487377933, 1.6121247434136865, 3.3613679248015504, 4.1316674342901845, 0.8303800120316661, -1.227825304579597, -1.8130593574910823, 0.1848756781930945, -0.676881418623572, 2.591255835558515, 1.6673593894980614, 1.6540892586338571, 1.6143400480675385, 0.23932506846152854, 1.5181285887362506, 0.13241105800686212, 3.8061670131492367, 0.5807166437788316, 1.5116799923186133, 1.5599307705774368, 0.29362851954048996, 0.19967375719855993, 1.914813488873402, 2.7644195885210587, 0.37942837286458875, 2.3383678677227993, -0.5843868818793704, 3.921194364357892, 1.4174799045629805, 2.741638785064822, -0.6304530862551285, -1.8933985338953436, 1.5624277556723982, -1.1677654133728248, -0.8250732415577517, 1.7230226879389658, -0.8418763059243636, 0.9691454037142504, 0.7252400407386729, -1.9797782989274328, 2.2739476528314704, 0.4612678359290186, 2.2453750405051833, 1.2218764611893127, -1.4926462511306318, 1.5083681953206398, -0.6209409896223363, -0.8575621399562154, -1.1360971020856898, -0.32413810704072976, 2.317045883444611, 0.052405537301035005]
# params_saved = {'I__n_0': 1.3926202901320248, 'I__n_1': 0.008448820382520305, 'I__n_2': -0.6385783774702594, 'aw__n.r@s_0': 0.004878974097219957, 'aw__n@n.l_0': -1.710437394121971, 'bad__n.r@s_0': -2.0120646244526674, 'bad__n@n.l_0': -0.33328110433838587, 'bland__n.r@s_0': -1.8011492489090937, 'bland__n@n.l_0': -0.23301002690494765, 'cook__n.r@s@n.l_0': 2.666641605278804, 'cook__n.r@s@n.l_1': 0.35782850997078663, 'delici__n.r@s_0': -0.6784353476839611, 'delici__n@n.l_0': 2.5048532581915994, 'dislik__n.r@s@n.l_0': 1.264688638132907, 'dislik__n.r@s@n.l_1': 2.9058201511277773, 'fast__n.r@s_0': 1.5228059027468268, 'fast__n@n.l_0': 1.4318617692717328, 'food__n_0': 0.9510109180885518, 'food__n_1': 1.983545943335132, 'food__n_2': 0.31422754989262236, 'good__n.r@s_0': -0.37898061288101936, 'good__n@n.l_0': 0.3417258364922475, 'great__n.r@s_0': 0.32023762451562343, 'great__n@n.l_0': 1.9344786256084243, 'had__n.r@s@n.l_0': 0.9175005505289897, 'had__n.r@s@n.l_1': -1.126491771734188, 'hate__n.r@s@n.l_0': 1.0178843262642405, 'hate__n.r@s@n.l_1': -0.8544213946932742, 'impecc__n.r@s_0': 0.8774940821788493, 'impecc__n@n.l_0': 0.28533233081922516, 'like__n.r@s@n.l_0': 0.5238711836690633, 'like__n.r@s@n.l_1': -0.0248482491004431, 'love__n.r@s@n.l_0': -1.0716522290529822, 'love__n.r@s@n.l_1': 0.4674574117431039, 'meal__n_0': -0.2913227630628624, 'meal__n_1': -0.7824158049259571, 'meal__n_2': 2.364685233940882, 'nice__n.r@s_0': -0.31906215984000647, 'nice__n@n.l_0': -2.087345064482271, 'restaur__n_0': 1.9567881690203595, 'restaur__n_1': 0.35319298096689367, 'restaur__n_2': -0.5953604557383299, 'rude__n.r@s_0': 2.1631540155864895, 'rude__n@n.l_0': -0.9753409914505806, 'servic__n_0': -2.7047864895209357, 'servic__n_1': 1.2351321590961786, 'servic__n_2': -1.5851261977616085, 'show__n.r@s@n.l_0': -0.6393012792612347, 'show__n.r@s@n.l_1': 1.7857303585103954, 'slow__n.r@s_0': 0.5156333465906835, 'slow__n@n.l_0': 1.0425765284092643, 'staff__n_0': 0.08543447326700565, 'staff__n_1': -2.068154595353262, 'staff__n_2': 0.019379227469981927, 'tasti__n.r@s_0': 1.2246133069435021, 'tasti__n@n.l_0': 1.9668107229444722, 'terribl__n.r@s_0': -0.3440262636975905, 'terribl__n@n.l_0': 2.1861199395142688, 'unappet__n.r@s_0': 1.440589048292482, 'unappet__n@n.l_0': -0.31598863954665035, 'wa__s@s.l_0': -1.0244128683696618}
spsa_a = 0.2
spsa_c = 0.06
spsa_n_iter = 0
k0 = 0

# SVM type
kernel = 'rbf'
default_svm = True
optimize_svm = False

# Experiment metadata
experiment_name = "misc"
verbose = True

In [ ]:
import os
import json

if (experiment_name == ""):
    experiment_name = "misc"

path = '../data/experiment_results/journal/generalQC/'+experiment_name

try:
    os.mkdir(path)
except:
    print("experiment folder already exists")

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import numpy as np

def read_data(filename):
    labels, sentences = [], []
    with open(filename) as f:
        for line in f:
            labels.append([1, 0] if line[0] == '1' else [0, 1])
            sentences.append(line[1:].strip())
    return np.array(labels), sentences

test_targets_src, test_data_src = read_data('../data/datasets/restaurant_v5_test.txt')
dev_targets_src, dev_data_src = read_data('../data/datasets/restaurant_v5_dev.txt')
train_targets_src, train_data_src = read_data('../data/datasets/restaurant_v5_train.txt')

In [ ]:
# Helper function for converting to bigraph

from discopy.rigid import Id as RigidId

def checkTrailingCups(diagram):
    scanWords = True
    
    for box in diagram.boxes:
        if not box.dom and not scanWords:
            return False
        else:
            scanWords = scanWords and not box.dom
    
    return True

def convertToTrailingCups(diagram):
    if (checkTrailingCups(diagram)):
        return diagram

    words = []
    cups = []
    
    for box in diagram.boxes:
        if not box.dom:
            words = words + [box]
        else:
            cups = [box] + cups
    
    new_diag = words[0]
    
    for i, word in enumerate(words):
        if i != 0:
            new_diag = new_diag >> RigidId(new_diag.cod) @ word
    
    for i, cup in enumerate(cups):
        if i != len(cups)-1:
            new_diag = new_diag >> RigidId(new_diag.cod[:-2]) @ cup
        else:
            new_diag = new_diag >> cup @ RigidId(new_diag.cod[2:])
    
    return new_diag

In [ ]:
# Fucntion for stemming and lemmatization of tokens

def to_word_tokens(data):
    return [word_tokenize(record) for record in data]

def build_stem_dictionary(data):
    port = PorterStemmer()
    wnet = WordNetLemmatizer()
    
    mapping = {}
    
    data_as_tokens = to_word_tokens(data)
    
    for words in data_as_tokens:
        for word in words:
            if word not in mapping:
                stemmed_word = port.stem(word)
                lemmatized_word = wnet.lemmatize(stemmed_word)
                
                mapping[word] = lemmatized_word
    
    return mapping

In [ ]:
# Function for stemming and lemmatization of diagram boxes

from lambeq.rewrite import RewriteRule

class StemRewriteRule(RewriteRule):
    def __init__(self, data):
        self.mapping = build_stem_dictionary(data)
    
    def matches(self, box):
        return box.name in self.mapping

    def rewrite(self, box):
        new_name = self.mapping[box.name]
        return type(box)(name=new_name, dom=box.dom, cod=box.cod)

In [ ]:
from lambeq.ansatz import CircuitAnsatz
from abc import abstractmethod
# from collections.abc import Mapping
from itertools import cycle
from typing import Callable, Optional, Tuple, Mapping
from discopy.quantum.circuit import (Circuit, Functor, Id, qubit)
from discopy.quantum.gates import Bra, Ket, Controlled, Rotation
from discopy.quantum.gates import H, X, Y, Z, Rx, Ry, Rz
from discopy.rigid import Box, Diagram, Ty
from discopy.tensor import Dim, Tensor
import numpy as np
from sympy import Symbol, symbols

class GeneralQCLayer(Circuit):
    
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import Rx, Rz

        if len(self.gate_layers) != 2:
            raise ValueError("Expected gate_layers as tuple of strings")
        
        g1, g2 = self.gate_layers

        if (g1 == None) or (g2 == None):
            raise ValueError("gate_layers must be in discopy.quantum.gates")
        # if not (abs(self.r1) == 1 and abs(self.r2) == 1) and ((abs(self.r1) == 1 or abs(self.r2) == 1) and (self.r2 % self.r1 == 0 or self.r1 % self.r2 == 0)) or (n_qubits % self.r1 == 0 and self.r1 != 1) or (n_qubits % self.r2 == 0 and self.r2 != 1):
        #     raise ValueError("n_qubits, r1, and r2 must be co-prime")

        params_shape = np.shape(params)

        if n_qubits == 1:
            if len(params) == 0:
                circuit = Id(1)
            else:
                circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
        elif (len(params_shape) != 2):
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits))
        else:
            g1_thetas = 0
            g2_thetas = 0

            if g1.free_symbols != {}:
                # g1 is fixed
                g1_thetas = n_qubits
            if g2.free_symbols != {}:
                g2_thetas = n_qubits

            if self.reuse_params:
                self.k = 1
            else:
                self.k = 2
            
            n_thetas = self.k*(g1_thetas + g2_thetas)

            if (params_shape[1] != n_thetas):
                raise ValueError(
                    "Expected component params with length {}".format(n_thetas))

            # ANSATZ ALGORITHM
            circuit = Id(n_qubits)
            
            # for {theta} in labelled params
            for thetas in params:
                
                # sublayer 1 non-entangling block
                if g1_thetas == 0:
                    # if g1 is fixed
                    sublayer1 = Id().tensor(*([g1 for _ in range(n_qubits)]))
                else:
                    # if g1 is trainable
                    sublayer1 = Id().tensor(*([g1(theta) for theta in thetas[:g1_thetas]]))
                
                # sublayer 1 entangled block
                ctrl = 0

                for i in range(n_qubits):
                    # shift target := control - r1
                    tgt = (ctrl - self.r1) % n_qubits
                    
                    if g2_thetas == 0:
                        sublayer1 = sublayer1._apply_controlled(g2, ctrl, tgt)
                        # sublayer1 = sublayer1.CX(ctrl, tgt)
                    else:
                        sublayer1 = sublayer1._apply_controlled(g2(thetas[g1_thetas + i]), ctrl, tgt)
                        # sublayer1 = sublayer1.CRx(thetas[g1_thetas + i], ctrl, tgt)
                    
                    ctrl = tgt
                
                # sublayer 2 non-entangling block
                if g1_thetas == 0:
                    sublayer2 = Id().tensor(*([g1 for _ in range(n_qubits)]))
                else:
                    if self.reuse_params:
                        sublayer2 = Id().tensor(*([g1(theta) for theta in thetas[:g1_thetas]]))
                    else:
                        sublayer2 = Id().tensor(*([g1(theta) for theta in thetas[g1_thetas+g2_thetas:2*g1_thetas+g2_thetas]]))
                
                # sublayer 2 entangled block
                ctrl = 0

                for i in range(n_qubits):
                    # shift target := control - r2
                    tgt = (ctrl - self.r2) % n_qubits

                    if g2_thetas == 0:
                        sublayer2 = sublayer2._apply_controlled(g2, ctrl, tgt)
                        # sublayer2 = sublayer2.CX(ctrl, tgt)
                    else:
                        if self.reuse_params:
                            sublayer2 = sublayer2._apply_controlled(g2(thetas[g1_thetas + i]), ctrl, tgt)
                        else:
                            sublayer2 = sublayer2._apply_controlled(g2(thetas[2*g1_thetas+g2_thetas+i]), ctrl, tgt)
                            # sublayer2 = sublayer2.CRx(thetas[2*g1_thetas+g2_thetas+i], ctrl, tgt)
                    
                    ctrl = tgt
            
                # compose circuit
                circuit >>= sublayer1 >> sublayer2
            
        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class GeneralQCAnsatz(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, gate_config = {"gate_layers": [H, Rx], "var_layers": 1}, r = (1, 3), reuse_params=True, discard: bool = False) -> None:
        self.gate_config = gate_config
        self.reuse_params = reuse_params
        self.r = r
        
        class GeneralQCInterface(GeneralQCLayer):
            """ Interface for GeneralQCLayer """

            def strToGate(gateStr):
                if gateStr == "H":
                    return H
                if gateStr == "X":
                    return X
                if gateStr == "Y":
                    return Y
                if gateStr == "Z":
                    return Z
                if gateStr == "Rx":
                    return Rx
                if gateStr == "Ry":
                    return Ry
                if gateStr == "Rz":
                    return Rz
                
                return Id(1)
            
            assert(len(self.gate_config["gate_layers"]) == 2)

            g1, g2 = self.gate_config["gate_layers"]

            if isinstance(g1, str):
                g1 = strToGate(g1)
            if isinstance(g2, str):
                g2 = strToGate(g2)

            gate_layers = [g1, g2]
            r1, r2 = self.r
            reuse_params = self.reuse_params

        super().__init__(ob_map, n_layers, n_single_qubit_params, GeneralQCInterface, discard, [Rx, Rz])

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        if self.reuse_params:
            k = 1
        else:
            k = 2
        
        return (self.n_layers, k*self.gate_config["var_layers"]*n_qubits)

In [ ]:
class SimpleSA(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, Rx, Rz, CX, Ry

        if n_qubits == 1:
            if len(params) == 0:
                circuit = Id(1)
            else:
                circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
        elif len(np.shape(params)) != 2\
                or np.shape(params)[1] != n_qubits - 1:
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits - 1))
        else:
            depth = np.shape(params)[0]
            circuit = Id(n_qubits)

            for thetas in params:
                hadamards = Id().tensor(*(n_qubits * [H]))
                rotations = Id(n_qubits).then(*(
                    (Id(i) @ CX @ Id(n_qubits - 2 - i)) >> (Id(i + 1) @ Rz(thetas[i]) @ Id(n_qubits - 2 - i) >> (Id(i + 1) @ H @ Id(n_qubits - 2 - i)))
                    for i in range(n_qubits - 1)))
                circuit >>= hadamards >> rotations

        super().__init__(circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class SimpleSAAnsatz(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, discard: bool = False) -> None:
        super().__init__(ob_map, n_layers, n_single_qubit_params, SimpleSA, discard, [Rx, Rz], H)

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        return (self.n_layers, n_qubits - 1)

In [ ]:
from lambeq import BobcatParser, Rewriter, remove_cups
from discopy import grammar
from discopy.quantum.gates import X, Z
from discopy import rigid

def sentences_to_circuits(sentences, ansatz, convert_bigraph=True, not_representation="X", all_sentences=None, return_valids_mask=True):
    ### SENTENCES TO DIAGRAMS ###

    if all_sentences is None:
        all_sentences = sentences
    
    # syntax tree parsing
    parser = BobcatParser()
    raw_diagrams = parser.sentences2diagrams([text.replace(" not ", " ") for text in sentences])

    # filter valid diagrams type S
    n_sent = len(sentences)

    valids_mask = np.array([d.cod.name == Ty('s').name for d in raw_diagrams])
    data = [sentences[i] for i in range(n_sent) if valids_mask[i]]
    use_diagrams = [raw_diagrams[i] for i in range(n_sent) if valids_mask[i]]

    # grammatical rewrite rules
    rewriter = Rewriter()
    rewritten_diagrams = [rewriter(diagram) for diagram in use_diagrams]

    # bigraph method
    normalised_diagrams = [convertToTrailingCups(diagram.normal_form()) for diagram in rewritten_diagrams]

    removed_diagrams = [remove_cups(diagram) for diagram in normalised_diagrams]

    # stemming and lemmatization
    stemmed_diagrams = [Rewriter([StemRewriteRule(all_sentences)])(diagram) for diagram in removed_diagrams]

    # final diagrams
    diagrams = [diagram for diagram in stemmed_diagrams]

    ### DIAGRAMS to CIRCUITS ###

    # string diagrams to raw quantum circuits
    circuits = [ansatz(diagram) for diagram in diagrams]

    # apply NOT box to circuits
    for i, circuit in enumerate(circuits):
        if data[i].find(" not ") != -1:
            if (not_representation == "ZX"):
                circuits[i] = circuit >> Z >> X
            else:
                circuits[i] = circuit >> X
    
    if return_valids_mask:
        return data, diagrams, circuits, valids_mask
    else:
        return data, diagrams, circuits

# Compile Experiments

In [ ]:
import os
import json

working_dir = "../data/experiment_results/journal/generalQC/"
experiment_book = []

for experiment_name in os.listdir(working_dir):
    experiment_dir = working_dir+experiment_name+"/"
    if os.path.isdir(experiment_dir):
        if "test_800_accs.csv" in os.listdir(experiment_dir) and "faulty" not in experiment_name:
            experiment_dict = {"name": experiment_name}

            with open(experiment_dir+"params.txt", "r") as f:
                content = f.read().replace("\'", "\"")
                params_dict = json.loads(content)
                f.close()
            
            experiment_dict["params"] = params_dict

            info = experiment_name.split("_")

            experiment_dict["data_version"] = info[0]

            if info[1] == "SimpleSA":
                experiment_dict["ansatz"] = "SimpleSAAnsatz"
            elif info[2] == "IQP":
                experiment_dict["ansatz"] = "IQPAnsatz"
            else:
                experiment_dict["ansatz"] = "GeneralQCAnsatz"

                experiment_dict["gate_layers"] = [info[1], info[2][1:]]

                experiment_dict["var_layers"] = (info[1]+info[2]).count("R")
            
            experiment_dict["n_layers"] = 1

            for i in info[1:]:
                if "iter" in i:
                    experiment_dict["iter"] = int(i[:-4])
                    break
            
            if "no" in info:
                experiment_dict["parameterize_nouns"] = False
            elif "with" in info:
                experiment_dict["parameterize_nouns"] = True
        
            experiment_book.append(experiment_dict)

In [ ]:
take = ["v5_IQP_800iter"]

experiment_book_2 = [e for e in experiment_book if e["name"] in take]

experiment_book = [e for e in experiment_book_2]

experiment_book

In [ ]:
from pytket.extensions.qiskit import AerBackend

backend = AerBackend()

backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'n_shots': 8192  # maximum recommended shots, reduces sampling error
}

In [ ]:
from discopy.quantum import Circuit, Id, Measure

def randint(rng, low=-1 << 63, high=1 << 63-1):
    return rng.integers(low, high)

def normalise(predictions):
    # apply smoothing to predictions
    predictions = np.abs(predictions) + 1e-9
    return predictions / predictions.sum()

def make_pred_fn(circuits, parameters, rng):
    measured_circuits = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuits]
    circuit_fns = [c.lambdify(*parameters) for c in measured_circuits]

    def predict(params):
        outputs = Circuit.eval(*(c_fn(*params) for c_fn in circuit_fns),
                               **backend_config, seed=randint(rng))
        return np.array([normalise(output.array) for output in outputs])
    return predict

In [ ]:
from lambeq import AtomicType, IQPAnsatz, Sim15Ansatz
from sympy import default_sort_key


def decapitate_circuit(circuit):
    from discopy import Bra, Measure

    cutoff = None

    for i, box in enumerate(circuit.boxes):
        if box in [Bra, Measure, Bra(0), Bra(0, 0)]:
            cutoff = i
            break
    
    return circuit[:i]

def build_circuit_library(experiment_dict, data_src, targets_src):
    name = experiment_dict["name"]
    params = experiment_dict["params"]
    ansatz_name = experiment_dict["ansatz"]
    n_layers = experiment_dict["n_layers"]
    spsa_n_iter = experiment_dict["iter"]
    parameterize_nouns = experiment_dict["parameterize_nouns"]
    
    parameters = list(params.keys())
    param_vals = [params[key] for key in parameters]

    # build ansatz
    N = AtomicType.NOUN
    S = AtomicType.SENTENCE

    if ansatz_name == "IQPAnsatz":
        ansatz = IQPAnsatz({N: 1, S: 1}, n_layers=n_layers)
    elif ansatz_name == "SimpleSAAnsatz":
        ansatz = SimpleSAAnsatz({N: 1, S: 1}, n_layers=n_layers, n_single_qubit_params=3*int(parameterize_nouns))
    elif ansatz_name == "GeneralQCAnsatz":
        gate_layers = experiment_dict["gate_layers"]
        var_layers = experiment_dict["var_layers"]
        
        ansatz = GeneralQCAnsatz({N: 1, S: 1}, n_layers=1, n_single_qubit_params=3*int(parameterize_nouns), gate_config={"gate_layers": gate_layers, "var_layers": var_layers}, r=(1, -1))
    
    data, diagrams, circuits, valids_mask = sentences_to_circuits(data_src, ansatz)
    targets = [target for i, target in enumerate(targets_src) if valids_mask[i]]

    circuit_heads = []

    # instantiate complete circuits

    SEED = 0
    rng = np.random.default_rng(SEED)

    use_parameters = sorted({s for circ in circuits for s in circ.free_symbols},key=default_sort_key)
    use_param_strs = [str(param_sym) for param_sym in use_parameters]
    use_param_vals = [val for i, val in enumerate(param_vals) if parameters[i] in use_param_strs]

    measured_circuits = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuits]
    circuit_fns = [c.lambdify(*use_parameters) for c in measured_circuits]
    instantiated_circuits = [c_fn(*use_param_vals) for c_fn in circuit_fns]

    outputs = Circuit.eval(*instantiated_circuits, **backend_config, seed=randint(rng))
    res = np.array([normalise(output.array) for output in outputs])

    circuit_heads = [decapitate_circuit(c) for c in circuits]

    # measured_circuit_heads = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuit_heads]
    circuit_head_fns = [c.lambdify(*use_parameters) for c in circuit_heads]
    instantiated_circuit_heads = [c_fn(*use_param_vals) for c_fn in circuit_head_fns]

    return data, targets, parameters, param_vals, diagrams, ansatz, circuits, instantiated_circuits, circuit_heads, instantiated_circuit_heads, outputs, res

In [ ]:
def computational_bases(n_qubits):
    return np.array([[int(j==i) for j in range(2**n_qubits)] for i in range(2**n_qubits)], dtype='cfloat')

def tensor_prod(v2, v1):
    return np.tensordot(v1, v2, axes=0).flatten()

def bloch_bases(n_qubits, n_alpha, n_beta):
    z_1 = computational_bases(1)
    b_plus = lambda alpha, beta: np.cos(alpha/2)*z_1[0] + np.sin(alpha/2)*np.exp((0+1j)*beta)*z_1[1]
    b_minus = lambda alpha, beta: np.sin(alpha/2)*z_1[0] - np.cos(alpha/2)*np.exp((0+1j)*beta)*z_1[1]

    alpha_dom = np.linspace(0, np.pi, n_alpha)
    beta_dom = np.linspace(0, 2*np.pi, n_beta)

    def b_bases_paramx(thetas):
        b_bases = computational_bases(0)

        for i in range(n_qubits):
            b_pluses = [tensor_prod(b_plus(thetas[2*i], thetas[2*i+1]), b_basis) for b_basis in b_bases]
            b_minuses = [tensor_prod(b_minus(thetas[2*i], thetas[2*i+1]), b_basis) for b_basis in b_bases]

            b_bases = np.vstack([b_pluses, b_minuses])
        
        return b_bases
    
    bloch_space = [None for _ in range((n_alpha*n_beta)**(n_qubits))]

    def recursive_search(level, curr_idx=[], curr_thetas=[]):
        for i, alpha0 in enumerate(alpha_dom):
            for j, beta0 in enumerate(beta_dom):
                if level == 1:
                    index = 0
                    for a_i, b_j in curr_idx+[[i, j]]:
                        index = index*n_alpha*n_beta+a_i*n_alpha+b_j
                    
                    bloch_space[index] = b_bases_paramx(curr_thetas+[alpha0, beta0])
                else:
                    recursive_search(level-1, curr_idx+[[i, j]], curr_thetas+[alpha0, beta0])

    recursive_search(n_qubits)

    return bloch_space

def trace(mat):
    return np.cfloat(sum([mat[i][i] for i in range(len(mat))]))

def abstrace(mat):
    return float(sum([np.absolute(mat[i][i]) for i in range(len(mat))]))

def KD(a, b, rho):
    Pi_a = np.outer(a, np.conjugate(a))
    Pi_b = np.outer(b, np.conjugate(b))
    return trace(np.matmul(Pi_b, np.matmul(Pi_a, rho)))

# Build Haar Basis

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

def build_clifford_set(n_qubits, cliffords=[]):
    single_qubit_cliffords = [
        '',
        'H', 'S',
        'HS', 'SH', 'SS',
        'HSH', 'HSS', 'SHS', 'SSH', 'SSS',
        'HSHS', 'HSSH', 'HSSS', 'SHSS', 'SSHS',
        'HSHSS', 'HSSHS', 'SHSSH', 'SHSSS', 'SSHSS',
        'HSHSSH', 'HSHSSS', 'HSSHSS'
    ]

    if n_qubits == 0:
        return cliffords
    else:
        if len(cliffords) == 0:
            return build_clifford_set(n_qubits-1, [[sc] for sc in single_qubit_cliffords])
        else:
            temp_cliffords = [[c+[sc] for sc in single_qubit_cliffords] for c in cliffords]
            new_cliffords = [item for sublist in temp_cliffords for item in sublist]

            return build_clifford_set(n_qubits-1, new_cliffords)

def build_clifford_states(n_qubits):
    backend = Aer.get_backend('unitary_simulator')

    multi_qubit_cliffords = build_clifford_set(n_qubits)
    mqc_svs = [clifford_str_to_sv(c, backend) for c in multi_qubit_cliffords]

    return mqc_svs

def clifford_str_to_sv(clifford_str, backend):
    n_qubits = len(clifford_str)

    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        for j, gate in enumerate(clifford_str[i]):
            if gate == "H":
                qc.h(n_qubits-i-1)
            if gate == "S":
                qc.s(n_qubits-i-1)
    
    qc.save_unitary()

    job = transpile(qc, backend)
    job_result = backend.run(job).result()

    return job_result.get_unitary(qc).data

In [ ]:
multi_qubit_cliffords = {}

for n_qubits in [2, 3, 4]:
    multi_qubit_cliffords[n_qubits] = build_clifford_states(n_qubits)

In [ ]:
import pickle

with open("../data/coherence_utils/mqc_vals.pkl", "wb") as f:
    pickle.dump(multi_qubit_cliffords, f)

In [ ]:
z_bases = {2: computational_bases(2), 3: computational_bases(3), 4: computational_bases(4)}

# Calculation

In [ ]:
import math
import time
from tqdm import tqdm

coherence_dict = {}

for k, experiment_dict in enumerate(experiment_book):
    if experiment_dict["name"] in coherence_dict:
        experiment_book[k]["NCLs"] = np.array(coherence_dict[experiment_dict["name"]]["ncls"])
        experiment_book[k]["coherence"] = coherence_dict[experiment_dict["name"]]["avg"]
        continue
    
    data, targets, parameters, param_vals, diagrams, ansatz, circuits, instantiated_circuits, circuit_heads, instantiated_circuit_heads, outputs, res = build_circuit_library(experiment_dict, test_data_src, test_targets_src)
    sv_data = [c.to_tk().get_statevector() for c in instantiated_circuit_heads]

    NCLs = [None for _ in sv_data]

    sv_lengths = [len(sv) for sv in sv_data]
    sv_sorted = [x for _, x in sorted(zip(sv_lengths, sv_data), key=lambda pair: pair[0])]
    sv_lengths_sorted = [x for x, _ in sorted(zip(sv_lengths, sv_data), key=lambda pair: pair[0])]

    dim = sv_lengths_sorted[0]
    i = 0

    start_time = time.time()

    while i < len(sv_data):
        i0 = i
        while i < len(sv_data) and sv_lengths_sorted[i] == dim:
            i += 1
        
        sv_use = sv_sorted[i0:i]
        print(experiment_dict["name"], dim, i)

        n_qubits = int(math.log2(dim))

        if n_qubits not in multi_qubit_cliffords:
            multi_qubit_cliffords[n_qubits] = build_clifford_states(n_qubits)
        if n_qubits not in z_bases:
            z_bases[n_qubits] = computational_bases(n_qubits)
        
        rhos = [np.outer(sv, sv.conjugate()) for sv in sv_use]

        y_max = np.array([-1 for _ in sv_use], dtype='float')

        for haar_basis in tqdm(multi_qubit_cliffords[n_qubits]):
            kd_sums = np.array([-1 for _ in sv_use], dtype='float')
            
            for hs in haar_basis:
                Pi_b = np.outer(hs, hs.conjugate())
                kd_sum = np.array([abstrace(np.matmul(Pi_b, rho)) for rho in rhos], dtype='float')
                kd_sums += kd_sum
                
            y_max = y_max*(y_max > kd_sums) + kd_sums*(kd_sums >= y_max)

        for j in range(i0, i):
            NCLs[j] = y_max[j-i0]
        if i < len(sv_data):
            dim = sv_lengths_sorted[i]

    end_time = time.time()

    experiment_book[k]["NCLs"] = NCLs
    experiment_book[k]["coherence"] = sum(NCLs)/len(NCLs)
    coherence_dict[experiment_dict["name"]] = {"name": NCLs, "avg": sum(NCLs)/len(NCLs)}

    print(experiment_dict["name"], NCLs, sum(NCLs)/len(NCLs))
    print(end_time-start_time)

In [ ]:
for e in experiment_book:
    print(e["name"])